## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Libraries</p>

In [9]:
import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import urllib.request as req
import json
from tqdm import tqdm, tqdm_notebook
import requests

import pandas as pd
import re
import string

from pandas.io.formats.style import Styler
import seaborn as sns

from itertools import chain

pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

tqdm.pandas()

rc = {
    "axes.facecolor": "#F8F8F8", 
    "figure.facecolor": "#F8F8F8", 
    "axes.edgecolor": "#000000",  
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000", 
    "ytick.color": "#000000",
    "grid.alpha": 0.4 
}

sns.set(rc=rc) 
palette = ['#ff7f50', '#ffd700', '#ffdab9', '#9fe2bf',
           '#d2b48c', '#008080', '#98ff98', '#000080']


from colorama import Style, Fore 
blk = Style.BRIGHT + Fore.BLACK
gld = Style.BRIGHT + Fore.YELLOW
grn = Style.BRIGHT + Fore.GREEN
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

In [10]:
def magnify(is_test : bool = False): 
    base_color = '#b57edc'
    if is_test:
        highlight_target_row = []
    else:
        highlight_target_row = dict(selector = 'tr:last-child',
                            props = [('background-color', f'{base_color}' + '20')]) 
    
    return [dict(selector="th", 
                props=[("font-size", "11pt"),
                    ('background-color', f'{base_color}'),
                    ('color', 'white'),
                    ('font-weight', 'bold'),
                    ('border-bottom', '0.1px solid white'), 
                    ('border-left', '0.1px solid white'), 
                    ('text-align', 'right')]),
        
            dict(selector='th.blank.level0', 
                props=[('font-weight', 'bold'),
                        ('border-left', '1.7px solid white'),
                        ('background-color', 'white')]),

            dict(selector="td", 
                    props=[('padding', "0.5em 1em"), 
                        ('text-align', 'right')]),

            dict(selector="th:hover",
                    props=[("font-size", "14pt")]),

            dict(selector="tr:hover td:hover",
                    props=[('max-width', '250px'),
                        ('font-size', '14pt'),
                        ('color', f'{base_color}'),
                        ('font-weight', 'bold'),
                        ('background-color', 'white'),
                        ('border', f'1px dashed {base_color}')]),
            
            dict(selector="caption", 
                props=[(('caption-side', 'bottom'))])] + highlight_target_row

def stylize_simple(df: pd.DataFrame, caption: str) -> Styler:
    """
        Args:
            df: any dataframe (train/test/origin)

        Returns:
            s: the dataframe wrapped into Styler.
    """
    s = df
    s = s.style.set_table_styles(magnify(True)).set_caption(f"{caption}")
    return s

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Intro</p>

Collecting Tokyo Real Estate Case Data

 <a href = 'http://www.oshimaland.co.jp'> 오시마랜드(Oshimaland)</a>
>This website showcases buildings with unfortunate past incidents such as suicide cases, corpse disposal incidents, and murder cases.


## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#4E4FEB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #4E4FEB">Data Crawling</p>

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#0B666A; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #0B666A">도쿄 다마 구역 추가 크롤링</p>


In [11]:
damaList = ['小金井市', '小平市', '狛江市', '国分寺市', '国立市', '清瀬市', '西東京市', '多摩市', '立川市', '町田市', '武藏野市', '武藏村山市', '三鷹市',
            'あきる野市','昭島市','青梅市', '稲城市', '調布市', '羽村市', '八王子市', '府中市', '東村山市', '東大和市', '東久留米市', '日野市']

In [18]:
jp_district_list = [
    '新宿区', '台東区', '墨田区', '豊島区', '渋谷区',
    '港区', '世田谷区', '大田区', '中野区', '中央区',
    '北区', '葛飾区', '杉並区', '江東区', '江戸川区',
    '文京区', '荒川区', '板橋区', '品川区', '千代田区',
    '足立区', '目黒区', '練馬区', '府中市', '八王子市',
    '日野市', '国分寺市', '町田市', '青梅市', '調布市',
    '武蔵野市', 'あきる野市', '三鷹市', '小金井市', '東村山市',
    '多摩市', '国立市', '狛江市', '西東京市', '立川市',
    '小平市', '羽村市', '武蔵村山市', '奥多摩町', '昭島市',
    '福生市'
]

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#0B666A; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #0B666A">이전 구역(jp_district_list)과 겹치지 않는 구역은 총 5개이다. </p>
 > '清瀬市', '武藏野市', '武藏村山市', '稲城市', '東大和市', '東久留米市'

In [22]:
result_list = [x for x in damaList if x not in jp_district_list]

In [23]:
def getJsonData(data):
    json_url = 'https://api.oshimaland.co.jp/map'
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Origin": "https://www.oshimaland.co.jp"
    }
    response = requests.post(json_url, headers=headers, json=data)
    json_data = response.json()
    return json_data


In [24]:
def dataList(data, json_data):
    jsonValueList = []
    origin = list(data.values())
    flatten_list = list(chain(*origin))
    for idx in range(len(flatten_list)):
        for idx2 in range(len(json_data['markers'][flatten_list[idx]])):
            jsonValueList.append(json_data['markers'][flatten_list[idx]][idx2]['key'])
    return jsonValueList

In [25]:
def getinfo(jsonValueList):
    getDataList = []
    for idx in tqdm_notebook(range(len(jsonValueList))):
        json_url = f'https://www.oshimaland.co.jp/d/{jsonValueList[idx]}.json'
        url = req.Request(json_url, headers={"User-Agent": "Mozilla/5.0"})
        code = req.urlopen(url)
        soup = BeautifulSoup(code, 'html.parser')
        json_data = json.loads(soup.text)
        getDataList.append({
            'info' : json_data['info'],
            'address' : json_data['ad'],
            'dt' : json_data['dt'],
            'cr' : json_data['cr'],
        })
        
    return pd.DataFrame(getDataList)

## <p style="font-family:JetBrains Mono; font-weight:normal; letter-spacing:2px; color:#0B666A; font-size:80%; text-align:left;padding: 0px; border-bottom: 3px solid #0B666A">각 구역 마다, json 파라미터 값이 모두 다르므로, 파라미터 값을 변경하면서 데이터를 가져와야 한다.</p>
network tap -> fetch/xhr -> map.json

In [28]:
#* '清瀬市'
data = {'keys': ["1330021120330333", "1330021120331220", "1330021120331222"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df1 = getinfo(jsonValueList)
display(stylize_simple(df1.head(4), '清瀬市'))

  0%|          | 0/6 [00:00<?, ?it/s]

,info,address,dt,cr
0,今建っている家の前の平屋で男性が孤独死。後にミイラ化。,東京都清瀬市中清戸二丁目718-55,2001年頃,令和5年3月14日
1,101号室 角部屋 告知事項有り,東京都清瀬市中里三丁目79-10 東和コーポ,令和4年6月2日,令和4年6月2日
2,ここに住んでた中学生が飛び降り死亡,東京都清瀬市中里五丁目2-8,,平成24年1月22日
3,告知事項ありの記載,東京都清瀬市中清戸四丁目909-2 グランディール 1階,不明,平成29年10月24日


In [31]:
#* '武藏野市'
data = {'keys' : ["1330021122111312", "1330021122111313", "1330021122111330", "1330021122111332"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df2 = getinfo(jsonValueList)
display(stylize_simple(df2.head(4), '武藏野市'))

  0%|          | 0/11 [00:00<?, ?it/s]

,info,address,dt,cr
0,401号室 自殺,東京都西東京市東伏見四丁目8-24ラ・フローラ武蔵野6,7月20日,平成30年7月28日
1,飛び降り自殺,東京都西東京市東伏見五丁目4-5,令和5年1月,令和5年5月14日
2,パレホームズ607 ※告知事項あり 不動産情報サイトからの情報,東京都西東京市東伏見五丁目4-1,2020年12月,令和3年1月14日
3,心理的瑕疵物件 不動産情報に記載,東京都練馬区関町北三丁目4-5 アイル関町 2階,時期不明,平成29年10月27日


In [33]:
#* '狛江市'
data = {'keys': ["1330021123022001", "1330021123022003"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df3 = getinfo(jsonValueList)
display(stylize_simple(df3.head(4), '狛江市'))

  0%|          | 0/9 [00:00<?, ?it/s]

,info,address,dt,cr
0,心理的瑕疵有 内装リフォーム済：2018年01月,東京都狛江市和泉本町一丁目36-3 狛江セントラルハイツ３号棟 8階,時期不明,平成30年5月22日
1,205号室 過労死,東京都狛江市岩戸北一丁目12-10,平成30年3月25日,令和元年10月8日
2,遺体発見,東京都狛江市和泉本町一丁目13-16,平成30年1月28日,平成30年1月29日
3,15階より 飛び降り自殺,東京都狛江市和泉本町一丁目36-1 狛江セントラルハイツ1号棟,平成20年6月21日,令和4年9月26日


In [36]:
#* '国分寺市'
data = {'keys': ["1330021122103100"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df4 = getinfo(jsonValueList)
display(stylize_simple(df4.head(4), '国分寺市'))

  0%|          | 0/7 [00:00<?, ?it/s]

,info,address,dt,cr
0,首吊り自殺,東京都国分寺市戸倉一丁目8-25,2021年5月,令和3年6月27日
1,火災により78歳男性が4日後に亡くなった,東京都国分寺市日吉町四丁目6-1,平成29年7月22日,平成30年1月24日
2,5階 角部屋 告知事項有り,東京都国分寺市戸倉一丁目8-25 PlaceK,令和4年10月6日,令和4年10月7日
3,6階 角部屋 告知事項有り,東京都国分寺市戸倉一丁目8-25 PlaceK,令和4年10月6日,令和4年10月7日


In [39]:
#* '国立市'
data = {'keys': ["1330021122103202"]}
json_data = getJsonData(data)
jsonValueList = dataList(data, json_data)
df5 = getinfo(jsonValueList)
display(stylize_simple(df5.head(4), '国立市'))

  0%|          | 0/4 [00:00<?, ?it/s]

,info,address,dt,cr
0,死体発見,東京都国立市富士見台二丁目46 UR国立富士見台第2団地2-5号棟506,平成27年9月22日,平成27年9月23日
1,友人が自殺しました。,東京都国立市富士見台二丁目46-4 第2団地5-110,平成28年7月10日,平成30年3月10日
2,遺体発見,東京都国立市富士見台二丁目四六丁目2-5-401,2020年6月１７日,令和2年7月7日
3,2階上がって左端の部屋、異臭により通報、浴室で男性の腐乱死体発見。体は溶け下半身は白骨化していた模様。男性はパニック障害者で孤高死でした。,東京都国立市富士見台二丁目43-2,2019年秋,令和2年7月12日


In [47]:
dataframeNameList = [f"df{idx}" for idx in range(1, len(result_list))]
result = pd.DataFrame()
for df in dataframeNameList:
    result = pd.concat([result, eval(df)])

In [48]:
result.rename(columns = {'dt' : 'occurred_date', 'cr' : 'recorded_date'}, inplace = True)
result.reset_index(drop=True, inplace=True)

In [49]:
#* \n, \r 제거
def remove_newlines(text):
    return text.replace('\n', ' ').replace('\r', ' ')

In [50]:
#* 특수문자 제거
#* string.punctuation에 있는 특수문자를 모두 제거
#* string.punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
# 특수문자를 제거하는 함수 정의
def remove_special_characters(text):
    # string.punctuation에 있는 특수문자를 빈 문자열('')로 대체하여 제거
    for char in text:
        if char in string.punctuation:
            text = text.replace(char, '')
    return text
result = result.applymap(remove_special_characters)
display(stylize_simple(result.head(4), 'Total Dataset'))

,info,address,occurred_date,recorded_date
0,今建っている家の前の平屋で男性が孤独死。後にミイラ化。,東京都清瀬市中清戸二丁目71855,2001年頃,令和5年3月14日
1,101号室 角部屋 告知事項有り,東京都清瀬市中里三丁目7910 東和コーポ,令和4年6月2日,令和4年6月2日
2,ここに住んでた中学生が飛び降り死亡,東京都清瀬市中里五丁目28,,平成24年1月22日
3,告知事項ありの記載,東京都清瀬市中清戸四丁目9092 グランディール 1階,不明,平成29年10月24日


In [51]:
# #* 특수문자 제거2
# def remove_special_characters2(text):
#     return text.replace('(', '').replace(')', '')

# result = result.applymap(remove_special_characters2)
# result
#! (, ) 제거가 안됨
#! 화살표도 존재

In [52]:
result.to_csv('oshimaland_dataset2.csv', index = False)